In [ ]:
# These are all of the import statements needed. The import statements are split up by 
# general python packages, D Wave specific packages, and Qiskit's software packages.

# General Python packages
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

# D Wave Ocean software packages
import dimod
from dimod import ExactSolver
from dwave.system import DWaveSampler, FixedEmbeddingComposite
from dwave.samplers import SimulatedAnnealingSampler

# Qiskit software packages
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, Session, EstimatorV2 as Estimator, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit.circuit.library import EfficientSU2, QAOAAnsatz

# 0 - Setup

## Variables 

In [ ]:
# These are the different test-cases instances and the related capacity for each instance. The 
# item sizes for each of the different bins were generated randomly, from 1 to the bin capacity
# for that size. To test a specific instance, set bins_current to the bin list and capacity_current
# to the capacity you want to test for your specific bin packing problem instance.

bins_small = [94, 6]
bins_medium = [94, 30, 18, 17, 43, 86, 24, 21, 75, 29]
bins_large = [94, 30, 18, 17, 43, 86, 24, 21, 75, 29, 72, 20, 83, 66, 52, 8, 92, 97, 57, 70, 32, 71, 9, 62, 95, 38, 88, 68, 19, 79]

capacity_small = 100
capacity_medium = 150
capacity_large = 200

bins_current = bins_large
capacity_current = capacity_large

In [ ]:
# These are the scalars for the QUBO model of the bin packing problem. Tweaking these numbers
# will only affect the QUBO matrix that the exact and quantum annealing solvers use. These
# values were set through experimentation.

alpha = 1 # minimze num boxes
beta = capacity_current * capacity_current # ensure each item has a home
gamma = 1 # weight can't go over bin capacity

In [ ]:
# Test Case Methods

# This is a method that tests and outputs the final, formated solutions of the different solvers. 
# It checks that every item in the item list is included, and makes sure that there 
# aren't any extra items or any duplicate items. This method prints out three test cases to make
# sure the solution is valid and prints out the final assortment of items in bins.

def print_test_cases(binary_variables):
    count = 0
    sorted_bin = []
    
    for x in binary_variables:
        count = count + x
    
    num_items = items

    ## Pre-processing to convert the binary variable solution and place each item
    ## in their respective bin
    for bin in range(items):
        current_bin = []
        
        for item in range(items):
            index = (num_items * bin) + item
            
            if (binary_variables[index] == 1): # if the item is in the current bin
                current_bin.append(item)            
        
        sorted_bin.append(current_bin)
    
    ## Test Cases
    
    print("Are there only X items?")
    print(count == len(bins_current))
    
    print("Is every item included?")
    squished_bin = []
    for bin in sorted_bin:
        for item in bin:
            squished_bin.append(bins_current[item])
    
    print(set(squished_bin) == set(bins_current))
    
    print("Are there any duplicates?")
    isDuplicate = False
    for item in squished_bin:
        if squished_bin.count(item) > 1:
            isDuplicate = True
    
    print(isDuplicate)
    print(sorted_bin)

# 1 - Brute Force QUBO solver

## Create matrix representation of BPP QUBO

In [ ]:
# This code generates a matrix representation of the Bin Packing Problem QUBO model.
# The output of this model is a qubo_array, with a size equal to the number of binary
# variables squared

items = len(bins_current)

qubo_array = np.zeros((items * items, items * items)) # assuming 1 box per item = max boxes, need to make sure this is a square matrix though

for i in range(items * items):
    #print(i % items)
    for j in range(items * items - i):
        item_one = i % items
        bin_one = int(i / items)
        item_two = (j + i)  % items
        bin_two = int((j + i) / items)
        weight = 0
       
        if (i == j + i): # on-diagonal
            weight += alpha - beta + (bins_current[item_two] * bins_current[item_two]) - (2 * capacity_current * bins_current[item_two])
        else: # off-diagonal
            if (item_one == item_two): # if items are the same
                weight += 2 * beta
            elif (bin_one == bin_two): # if bins are the same
                weight += 2 * gamma * bins_current[item_one] * bins_current[item_two]
        
        qubo_array[i, j + i] = weight
            
print(qubo_array)

## Use the ExactSolver() to find the best solution

In [ ]:
# This code uses the exact solver and uses the QUBO matrix to find all of the different solutions 
# of the QUBO matrix with their corresponding energy levels.

sampler = ExactSolver()  

print("Converting QUBO to BQM")
bqm = dimod.BinaryQuadraticModel(qubo_array, "BINARY")

print("Optimizing!")
samples = sampler.sample(bqm)

print("Finished optimization!")

# the solution list, converted into a list of binary variables (0 or 1) representing if an item i is in bin j
value_list = [(int)(value) for value in samples.first.sample.values()] 

In [ ]:
# This prints out the test cases for the exact solver solution, allowing you to see if the solution 
# (taken from the solution with the least energy from the exact solver) 
# is valid.

print_test_cases([(int)(value) for value in samples.first.sample.values()])

# 2 - Quantum Annealing Approach

## Use the SimulatedAnnealingSampler() to simulate how quantum annealing would solve the QUBO

In [ ]:
# This code uses the annealing simulator with the QUBO matrix to find the optimal solution for the QUBO matrix in 
# the same way D Wave's quantum annealers would.

sampler = SimulatedAnnealingSampler()  

print("Converting QUBO to BQM")
bqm = dimod.BinaryQuadraticModel(qubo_array, "BINARY")

print("Optimizing!")
samples = sampler.sample(bqm, num_counts=20)

print("Finished optimization!")

In [ ]:
# This prints out the test cases for the quantum annealing simulator, allowing you to see if the solution 
# (taken from the solution with the least energy from the exact solver) 
# is valid.

print_test_cases(samples.record[0][0])

# 3 - Quantum Variational Approach

## Define scalars for testing

In [ ]:
# These are the scalars for the Ising Hamiltonian model of the Bin Packing Problem QUBO. 
# These are necessarily because of the shift in variables (binary to spin) from the 
# QUBO model to an Ising Hamiltonian model. Tweaking these numbers will only affect the
# QUBO matrix that the quantum variational approach and QAOA algorithm use. These values
# were set through experimentation.

a = 1
b = beta
c = 1

## Generate the Ising Hamiltonian of a bin packing problem instance

In [ ]:
# This method generates the Ising Hamiltonian pauli list that the quantum variational approach and QAOA algorithm. 
# The Pauli List is the format of the hamiltonian that IBM Qiskit uses (instead of the matrix form that the QUBO 
# model uses). This method returns a pauli-list, with the observables and their corresponding cofficients. 

def generate_ising_hamiltonian(bins_current):
    items = len(bins_current)
    total_variables = items * items
    total_weight = sum(bins_current)
    
    pauli_list = []
    # individual terms
    for i in range(items * items):
        #print(i % items)
        for j in range(items * items - i):
            item_one = i % items
            bin_one = int(i / items)
            item_two = (j + i)  % items
            bin_two = int((j + i) / items)
    
            if (i == j + i): # on-diagonal
                paulis = ["I"] * total_variables
                paulis[i] = "Z"
        
                weight = (a + (c * bins_current[item_one] * (total_weight - (2 * capacity_current)))) / 2
                pauli_list.append(("".join(paulis)[::-1], weight))
            else: # off-diagonal
                if (item_one == item_two):
                    paulis = ["I"] * total_variables
                    paulis[i], paulis[j + i] = "Z", "Z"
        
                    weight = b / 2
                    pauli_list.append(("".join(paulis)[::-1], weight))
                elif (bin_one == bin_two):
                    paulis = ["I"] * total_variables
                    paulis[i], paulis[j + i] = "Z", "Z"
        
                    weight = (c * bins_current[item_one] * bins_current[item_two]) / 2
                    pauli_list.append(("".join(paulis)[::-1], weight))

    return pauli_list

In [ ]:
# This is the cost hamiltonian that the quantum anzatz attempts to optimize.

cost_hamiltonian = SparsePauliOp.from_list(generate_ising_hamiltonian(bins_current))

## Set the Ansatz

In [ ]:
# This code uses the EfficientSU2 anzatz to optimize the Ising Hamiltonian of the BPP.

circuit = EfficientSU2(cost_hamiltonian.num_qubits)
circuit.measure_all()

circuit.decompose().draw('mpl')

# This sets the number of parameters. It will be useful when we need to know how many parameters to randomly generate.
num_params = circuit.num_parameters 

## Process the quantum circuit to run on a quantum computer

In [ ]:
# This code connects to IBM's servers to run the quantum anzatz circuit on an actual quantum computer. 
# It generates a backend optimized version of the circuit and hamiltonian, so that the circuit can run,
# using the native architecture and gates of the quantum computer backend.
# NOTE - NEEDS AN API TOKEN TO RUN

QiskitRuntimeService.save_account(channel="ibm_quantum", token="INSERT_API_TOKEN", overwrite=True, set_as_default=True)

service = QiskitRuntimeService(channel='ibm_quantum')
backend = service.least_busy(min_num_qubits=127)
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

ansatz_isa = pm.run(circuit)
hamiltonian_isa = cost_hamiltonian.apply_layout(layout=ansatz_isa.layout)

In [ ]:
# Return estimate of energy from estimator

#    Parameters:
#        params (ndarray): Array of ansatz parameters
#        ansatz (QuantumCircuit): Parameterized ansatz circuit
#        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
#        estimator (EstimatorV2): Estimator primitive instance
#        cost_history_dict: Dictionary for storing intermediate results

#    Returns:
#        float: Energy estimate
#    

def cost_func(params, ansatz, hamiltonian, estimator):
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

In [ ]:
# This code optimizes over the parameters of the quantum ansatz, using the Powell 
# optimizer. It returns the most optimal parameters.

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

# generates random parameters
x0 = 2 * np.pi * np.random.random(num_params)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 10

    res = minimize(
        cost_func,
        x0,
        args=(ansatz_isa, hamiltonian_isa, estimator),
        method="Powell",
    )

In [ ]:
# This plots the results of the optimization, showing the 
# cost of the solution over the iterations.

fig, ax = plt.subplots()
ax.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"])
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")
plt.draw()

In [ ]:
# This code runs the quantum ansatz one last time with the optimized parameters to get 
# the optimal solution to the Bin Packing problem instance. 

optimized_circuit = ansatz_isa.assign_parameters(res.x)

# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

pub= (optimized_circuit, )
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val/shots for key, val in counts_int.items()}
final_distribution_bin = {key: val/shots for key, val in counts_bin.items()}
print(final_distribution_int)

In [ ]:
# This code processes the output of running the quantum ansatz. It is an auxiliary 
# function to sample the most likely bitstring.

def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]

keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, total_variables)
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

In [ ]:
## Run test cases on quantum variational algorithm solution
print_test_cases(most_likely_bitstring)

# 4 - Quantum Approximate Optimization Algorithm

In [ ]:
# This is the cost hamiltonian that the QAOA algorithm attempts to optimize.

cost_hamiltonian = SparsePauliOp.from_list(generate_ising_hamiltonian(bins_current))

In [ ]:
# This code constructs the QAOA circuit in Qiskit from stratch. It takes in a cost hamiltonian, 
# in the form of a Sparce Pauli Operator list, and outputs a QAOA circuit, with a total of two 
# runs. The number of runs is tweakable, however.

## This method converts a Sparce Pauli operator into a set of qubit indicies. It returns the
## qubit index (or indicies) that the SparcePauliOperator applied a Z gate on. This is used 
## in the cost circuit to know where to place RZ and CRZ gates.
def find_indexes(string, letter):
    return [i for i, char in enumerate(string) if char == letter]

## These are the initial parameters that the QAOA starts of with.
initial_gamma = np.pi
initial_beta = np.pi/2
init_params = [initial_gamma, initial_beta, initial_gamma, initial_beta]

## Initializes the QAOA circuit
circuit = QuantumCircuit(total_variables, total_variables)

## Sets every qubit in superposition
for qubit in range(total_variables):
    circuit.h(qubit)

## This method takes in a circuit and a parameter to apply the cost layer of the QAOA function
## on the circuit. It uses the cost hamiltonian coefficients to mark the rotation of the RZ or
## CRZ gates and ends by applying a completed cost layer onto the passed in circuit. There is a 
## barrier gate at the start of the layer to assist with visulization.
def cost_circuit(circuit, gamma):
    circuit.barrier()
    
    index = 0

    for i in range(len(cost_hamiltonian.paulis)):
        indexes = find_indexes((str)(cost_hamiltonian.paulis[i]), 'Z')
        curr_weight = cost_hamiltonian.coeffs[i]
        
        if len(indexes) == 1:
            circuit.rz((int)(cost_hamiltonian.coeffs[indexes[0]]) * gamma, indexes[0])
        elif len(indexes) == 2:
            circuit.crz((int)(cost_hamiltonian.coeffs[indexes[0]]) * (int)(cost_hamiltonian.coeffs[indexes[1]]) * gamma, indexes[0], indexes[1])
    
## This method takes in a circuit and a parameter to apply the mixer layer of the QAOA function
## on the circuit. It ends by adding a RX gate to every qubit in the circuit. There is a barrier
## gate at the start of the layer to assist with visulization.
def mixer_circuit(circuit, beta):
    circuit.barrier()
    
    for qubit in range(total_variables):
        circuit.rx(2 * beta, qubit) 

## Sets the number of runs and the parameters in the cost and mixer function
num_runs = 2
gamma_list = ParameterVector("gamma", num_runs)
beta_list = ParameterVector("beta", num_runs)

## Adds a set number of cost and mixer layers, set as the number of runs
for run in range(num_runs):
    cost_circuit(circuit, gamma_list[run])
    mixer_circuit(circuit, beta_list[run])

## Measures all of the qubits
circuit.measure_all()

## Draws the circuit
circuit.draw('mpl')

In [ ]:
# This code connects to IBM's servers to run the quantum anzatz circuit on an actual quantum computer. 
# It generates a backend optimized version of the circuit and hamiltonian, so that the circuit can run,
# using the native architecture and gates of the quantum computer backend.
# NOTE - NEEDS AN API TOKEN TO RUN

service = QiskitRuntimeService(channel='ibm_quantum')
backend = service.least_busy(min_num_qubits=127)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3,
                                    backend=backend)

candidate_circuit = pm.run(circuit)

In [ ]:
# Return estimate of energy from estimator

#    Parameters:
#        params (ndarray): Array of ansatz parameters
#        ansatz (QuantumCircuit): Parameterized ansatz circuit
#        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
#        estimator (EstimatorV2): Estimator primitive instance
#        cost_history_dict: Dictionary for storing intermediate results

#    Returns:
#        float: Energy estimate
#    

def cost_func_estimator(params, ansatz, hamiltonian, estimator):

    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [ ]:
# This code optimizes over the parameters of the quantum ansatz, using the Powell 
# optimizer. It returns the most optimal parameters.

objective_func_vals = [] # Global variable

with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="Powell",
        tol=1e-2,
    )
    print(result)

In [ ]:
# This plots the results of the optimization, showing the 
# cost of the solution over the iterations.

plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

In [ ]:
# This code runs the quantum ansatz one last time with the optimized parameters to get 
# the optimal solution to the Bin Packing problem instance. 

optimized_circuit = candidate_circuit.assign_parameters(result.x)

# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

pub= (optimized_circuit, )
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val/shots for key, val in counts_int.items()}
final_distribution_bin = {key: val/shots for key, val in counts_bin.items()}
print(final_distribution_int)

In [ ]:
# This code processes the output of running the quantum ansatz. It is an auxiliary 
# function to sample the most likely bitstring.

def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]

keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, total_variables)
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

In [ ]:
# Run test cases on the QAOA solution

print_test_cases(most_likely_bitstring)